# Week 6 -- Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings
warnings.simplefilter('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Functions

In [3]:
def check_names(df1, df2):
    """
    This funciton compares the list of names from one dataframe to another and updates where the names do not match 
    so they will merge correctly.
    
    Parameters:
        df1: first dataframe to compare to; this should be the dataframe on the left that you will join with when 
        they merge.
        df2: second dataframe, should be the dataframe on the right for the later merge.
        
    Returns:
        Updated name list for df2 that should match df1.
    """
    
    df_players = list(df1['Player'])    
    for i in range(len(df2['Player'])):
        name = df2['Player'][i]
        split_name = name.split(' ')
        if (split_name[0] + ' ' + split_name[1]) in df_players:
            df2['Player'][i] = split_name[0] + ' ' + split_name[1]
        else:
            df2['Player'][i] = name

In [4]:
def name_update(dataframe, column):
    """
    This function updates a column in a dataframe by stripping any excess spaces surrounding the observation.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column: the column you want to update.
    
    Returns:
        Updated dataframe column.    
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip())

In [5]:
def update_int_dtype(dataframe, column_list):
    """
    This function updates the data type for a list of columns to integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column data types to integer.   
    """
    for column in column_list:
        dataframe[column] = dataframe[column].map(lambda x: int(x))

In [6]:
def update_float_dtype(dataframe, column_list):
    """
    This function takes a dataframe object and converts it to a float.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
    
    Returns:
        Updated dataframe column data types to a float.  
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = '0.0'
            else:
                dataframe[column][i] = item 
            updated_item = str(dataframe[column][i])
            ones, tenths = updated_item.split('.')
            ones = int(ones)
            tenths = int(tenths) * .1
            dataframe[column][i] = ones + tenths
        dataframe[column] = dataframe[column].map(lambda x: float(x))

In [7]:
def fill_blanks(dataframe, column_list):
    """
    This function fills blank values with 0. This should be used for a column that will be an integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.    
    
    Returns:
        Updated dataframe columns with filled values.      
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = 0
            else:
                dataframe[column][i] = item

In [8]:
def remove_comma(dataframe, column_list):
    """
    This function removes the comma from a value in a column.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns less any commas that might appear.    
    """  
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item.split(',')) > 1:
                one, two = item.split(',')
                dataframe[column][i] = one + two

In [9]:
def adjust_float(dataframe, column_list):
    """
    This function adjusts values in a column to be workable with the 'update_float_dtype' function.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns with objects with a decimal point value behind it.    
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item) < 3:
                dataframe[column][i] = item + '.0'
            else:
                dataframe[column][i] = item

In [10]:
def update_percent(dataframe, column):
    """
    This function updates a column with a percent sign so it can be manipulated into a float with the 
    'update_float_dtype' function.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed.     
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip('%'))
    for i in range(len(dataframe[column])):
        item = dataframe[column][i]
        if len(item) < 3:
            dataframe[column][i] = item + '.0'
        else:
            dataframe[column][i] = item

In [11]:
def replace_dash(dataframe, column_list, dash):
    """
    This function replaces a - in a dataframe so datatypes can then be updated.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        dash: set to a value in a specific column in the dataframe.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed. 
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == dash:
                dataframe[column][i] = '0'
            else:
                dataframe[column][i] = item

## Read in Collected Data

In [33]:
df = pd.read_pickle('player_stats')
fantasy_pts = pd.read_pickle('fantasy_weeks')
defense = pd.read_pickle('defense_data')
kicking = pd.read_pickle('kicking_data')

## Merge Last Week Fantasy Pts with this Week

In [34]:
no_games = 6

In [35]:
lastwk = pd.read_pickle('fantweeks_1_' + str(no_games - 1))
lastwk.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5
0,Russell Wilson,SEA,QB,150.4,31.8,34.4,36.8,21.9,25.5
1,Patrick Mahomes II,KC,QB,138.8,20.4,27.5,40.0,20.2,30.7
2,Josh Allen,BUF,QB,138.6,28.2,34.5,32.2,25.4,18.3
3,Dak Prescott,DAL,QB,138.6,17.6,39.8,29.5,38.3,13.4
4,Kyler Murray,ARI,QB,135.5,27.3,33.1,24.7,23.1,27.3


In [36]:
fantasy_pts = pd.merge(fantasy_pts, lastwk, 'left', on='Player')
fantasy_pts.head()

,Player,Team_x,Position_x,TTL_x,Week_6,Team_y,Position_y,TTL_y,Week_1,Week_2,Week_3,Week_4,Week_5
0,Kyler Murray,ARI,QB,164.4,28.9,ARI,QB,135.5,27.3,33.1,24.7,23.1,27.3
1,Patrick Mahomes II,KC,QB,159.4,20.6,KC,QB,138.8,20.4,27.5,40.0,20.2,30.7
2,Josh Allen,BUF,QB,154.7,16.1,BUF,QB,138.6,28.2,34.5,32.2,25.4,18.3
3,Russell Wilson,SEA,QB,150.4,NaN,SEA,QB,150.4,31.8,34.4,36.8,21.9,25.5
4,Dak Prescott,DAL,QB,138.6,NaN,DAL,QB,138.6,17.6,39.8,29.5,38.3,13.4


In [37]:
fantasy_pts.drop(columns=['Team_y', 'Position_y', 'TTL_y'], inplace=True)
fantasy_pts.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position', 'TTL_x': 'TTL'}, inplace = True)
fantasy_pts.columns

Index(['Player', 'Team', 'Position', 'TTL', 'Week_6', 'Week_1', 'Week_2',
       'Week_3', 'Week_4', 'Week_5'],
      dtype='object')

In [38]:
fantasy_pts = fantasy_pts[['Player', 'Team', 'Position', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 
                           'Week_6']]
fantasy_pts.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Kyler Murray,ARI,QB,164.4,27.3,33.1,24.7,23.1,27.3,28.9
1,Patrick Mahomes II,KC,QB,159.4,20.4,27.5,40.0,20.2,30.7,20.6
2,Josh Allen,BUF,QB,154.7,28.2,34.5,32.2,25.4,18.3,16.1
3,Russell Wilson,SEA,QB,150.4,31.8,34.4,36.8,21.9,25.5,NaN
4,Dak Prescott,DAL,QB,138.6,17.6,39.8,29.5,38.3,13.4,NaN


In [39]:
print(fantasy_pts.shape)

(617, 10)


### Pickle Combined DataFrame for Next Week

In [40]:
fantasy_pts.to_pickle('fantweeks_1_' + str(no_games))

## Merge Offensive Players & Weekly Fantasy Pts

In [41]:
print(df.shape)
df.head()

(518, 28)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank
0,Alvin Kamara,NOR,RB,25,5,4,0,0,0,0,0,61,281,4.61,4,45,38,395,10.39,3,0,0,7,,,128.6,1,1
1,Derrick Henry,TEN,RB,26,5,5,0,0,0,0,0,123,588,4.78,6,15,8,84,10.50,0,0,0,6,,,107.2,2,2
2,Kyler Murray,ARI,QB,23,6,6,135,205,1487,10,6,51,370,7.25,6,0,0,0,,0,1,1,6,,,164.5,1,3
3,Patrick Mahomes,KAN,QB,25,6,6,144,219,1699,15,1,34,165,4.85,2,0,0,0,,0,1,0,2,,2,159.5,2,4
4,Dalvin Cook,MIN,RB,25,5,5,0,0,0,0,0,92,489,5.32,7,16,12,64,5.33,0,1,1,7,3,,107.3,3,5


In [42]:
print(fantasy_pts.shape)
fantasy_pts.head()

(617, 10)


,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Kyler Murray,ARI,QB,164.4,27.3,33.1,24.7,23.1,27.3,28.9
1,Patrick Mahomes II,KC,QB,159.4,20.4,27.5,40.0,20.2,30.7,20.6
2,Josh Allen,BUF,QB,154.7,28.2,34.5,32.2,25.4,18.3,16.1
3,Russell Wilson,SEA,QB,150.4,31.8,34.4,36.8,21.9,25.5,NaN
4,Dak Prescott,DAL,QB,138.6,17.6,39.8,29.5,38.3,13.4,NaN


In [43]:
check_names(df, fantasy_pts)

In [44]:
name_update(df, 'Player')
name_update(fantasy_pts, 'Player')

In [45]:
offense = pd.merge(df, fantasy_pts, 'left', on='Player')

In [46]:
offense.drop(columns = ['Team_y', 'Position_y'], inplace = True)
offense.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position'}, inplace = True)

In [47]:
print(offense.shape)
offense.head(10)

(518, 35)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Alvin Kamara,NOR,RB,25,5,4,0,0,0,0,0,61,281,4.61,4,45,38,395,10.39,3,0,0,7,,,128.6,1,1,109.6,18.7,29.4,31.7,17.9,11.9,NaN
1,Derrick Henry,TEN,RB,26,5,5,0,0,0,0,0,123,588,4.78,6,15,8,84,10.50,0,0,0,6,,,107.2,2,2,103.2,13.1,8.4,25.0,NaN,18.3,38.4
2,Kyler Murray,ARI,QB,23,6,6,135,205,1487,10,6,51,370,7.25,6,0,0,0,,0,1,1,6,,,164.5,1,3,164.4,27.3,33.1,24.7,23.1,27.3,28.9
3,Patrick Mahomes,KAN,QB,25,6,6,144,219,1699,15,1,34,165,4.85,2,0,0,0,,0,1,0,2,,2,159.5,2,4,159.4,20.4,27.5,40.0,20.2,30.7,20.6
4,Dalvin Cook,MIN,RB,25,5,5,0,0,0,0,0,92,489,5.32,7,16,12,64,5.33,0,1,1,7,3,,107.3,3,5,101.3,20.8,15.1,23.9,26.6,14.9,NaN
5,Adam Thielen,MIN,WR,30,6,6,0,0,0,0,0,3,15,5.00,0,49,32,415,12.97,7,0,0,7,1,,103.0,1,6,87.0,25.0,3.1,9.3,18.2,20.3,11.1
6,Aaron Jones,GNB,RB,26,5,5,0,0,0,0,0,75,389,5.19,5,28,18,161,8.94,2,1,0,7,,,106.0,4,7,97.0,13.6,41.6,14.6,17.1,NaN,10.1
7,Calvin Ridley,ATL,WR,26,6,6,0,0,0,0,0,3,14,4.67,0,57,35,546,15.60,5,0,0,5,,,103.5,2,8,86.0,24.9,22.9,11.7,0.0,13.6,12.9
8,Josh Allen,BUF,QB,24,6,6,145,216,1711,16,4,37,143,3.86,3,0,0,0,,0,3,3,3,,,154.7,3,9,154.7,28.2,34.5,32.2,25.4,18.3,16.1
9,Travis Kelce,KAN,TE,31,6,6,0,0,0,0,0,0,0,,0,53,37,470,12.70,5,1,1,5,,,93.5,1,10,75.0,11.0,15.0,8.7,7.0,16.8,16.5


In [48]:
offense.isnull().sum()

Player                     0
Team                       0
Position                   0
Age                        0
Games                      0
GamesStarted               0
CompletedPasses            0
PassesAttempted            0
PassingYds                 0
PassingTDs                 0
Interceptions              0
RushingAttempts            0
RushingYds                 0
RushingYdspAtt             0
RushingTDs                 0
Targeted                   0
Receptions                 0
ReceivingYds               0
YdspReception              0
ReceivingTDs               0
Fumbles                    0
LostFumbles                0
TtlTDs                     0
TwoPTConversions           0
TwoPTConversionPasses      0
FDFantasyPts               0
PositionRank               0
OverallRank                0
TTL                       10
Week_1                   111
Week_2                   113
Week_3                   114
Week_4                   141
Week_5                   170
Week_6        

In [49]:
offense.fillna('0.0', inplace = True)

In [50]:
off_integers = ['Age', 'Games', 'GamesStarted', 'CompletedPasses', 'PassesAttempted', 'PassingYds', 'PassingTDs', 
            'Interceptions', 'RushingAttempts', 'RushingYds', 'RushingTDs', 'Targeted', 'Receptions', 
            'ReceivingYds', 'ReceivingTDs', 'Fumbles', 'LostFumbles', 'TtlTDs']

off_floats = ['RushingYdspAtt', 'YdspReception', 'FDFantasyPts', 'TTL','Week_1', 'Week_2', 'Week_3', 'Week_4', 
              'Week_5', 'Week_6']

In [51]:
update_int_dtype(offense, off_integers)

In [52]:
two_pts = ['TwoPTConversions', 'TwoPTConversionPasses']
fill_blanks(offense, two_pts)
update_int_dtype(offense, two_pts)

In [53]:
update_float_dtype(offense, off_floats)

In [54]:
offense.head()

,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Alvin Kamara,NOR,RB,25,5,4,0,0,0,0,0,61,281,10.1,4,45,38,395,13.9,3,0,0,7,0,0,128.6,1,1,109.6,18.7,29.4,31.7,17.9,11.9,0.0
1,Derrick Henry,TEN,RB,26,5,5,0,0,0,0,0,123,588,11.8,6,15,8,84,15.0,0,0,0,6,0,0,107.2,2,2,103.2,13.1,8.4,25.0,0.0,18.3,38.4
2,Kyler Murray,ARI,QB,23,6,6,135,205,1487,10,6,51,370,9.5,6,0,0,0,0.0,0,1,1,6,0,0,164.5,1,3,164.4,27.3,33.1,24.7,23.1,27.3,28.9
3,Patrick Mahomes,KAN,QB,25,6,6,144,219,1699,15,1,34,165,12.5,2,0,0,0,0.0,0,1,0,2,0,2,159.5,2,4,159.4,20.4,27.5,40.0,20.2,30.7,20.6
4,Dalvin Cook,MIN,RB,25,5,5,0,0,0,0,0,92,489,8.2,7,16,12,64,8.3,0,1,1,7,3,0,107.3,3,5,101.3,20.8,15.1,23.9,26.6,14.9,0.0


In [55]:
team_dict = {'LAR': 'Los Angeles Raiders', 
             'SEA': 'Seattle Seahawks', 
             'BAL': 'Baltimore Ravens', 
             'KAN': 'Kansas City Chiefs', 
             'JAX': 'Jacksonville Jaguars', 
             'DET': 'Detroit Lions', 
             'PIT': 'Pittsburgh Steelers', 
             'MIN': 'Minnesota Vikings', 
             'CHI': 'Chicago Bears', 
             'ATL': 'Atlanta Falcons', 
             'BUF': 'Buffalo Bills', 
             'GNB': 'Green Bay Packers', 
             'MIA': 'Miami Dolphins', 
             'CLE': 'Cleveland Browns', 
             'CAR': 'Carolina Panthers', 
             'DAL': 'Dallas Cowboys', 
             'ARI': 'Arizona Cardinals', 
             'HOU': 'Houston Texans', 
             'NYG': 'New York Giants', 
             'WAS': 'Washington Football Team', 
             'IND': 'Indianapolis Colts', 
             'LVR': 'Las Vegas Raiders', 
             'DEN': 'Denver Broncos', 
             'TEN': 'Tennessee Titans', 
             'NWE': 'New England Patriots', 
             'CIN': 'Cincinnati Bengals', 
             'NYJ': 'New York Jets', 
             'LAC': 'Los Angeles Chargers', 
             'PHI': 'Philadelphia Eagles', 
             'TAM': 'Tampa Bay Buccaneers', 
             'NOR': 'New Orleans Saints', 
             'SFO': 'San Francisco 49ers'}

team_names = pd.DataFrame()
team_names['Team'] = list(team_dict.keys())
team_names['Long_Name'] = list(team_dict.values())
team_names.head()

,Team,Long_Name
0,LAR,Los Angeles Raiders
1,SEA,Seattle Seahawks
2,BAL,Baltimore Ravens
3,KAN,Kansas City Chiefs
4,JAX,Jacksonville Jaguars


In [56]:
offense.dtypes

Player                    object
Team                      object
Position                  object
Age                        int64
Games                      int64
GamesStarted               int64
CompletedPasses            int64
PassesAttempted            int64
PassingYds                 int64
PassingTDs                 int64
Interceptions              int64
RushingAttempts            int64
RushingYds                 int64
RushingYdspAtt           float64
RushingTDs                 int64
Targeted                   int64
Receptions                 int64
ReceivingYds               int64
YdspReception            float64
ReceivingTDs               int64
Fumbles                    int64
LostFumbles                int64
TtlTDs                     int64
TwoPTConversions           int64
TwoPTConversionPasses      int64
FDFantasyPts             float64
PositionRank              object
OverallRank               object
TTL                      float64
Week_1                   float64
Week_2    

## Defense Stats

In [57]:
defense.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks
0,Pittsburgh Steelers,5,94,294,4.9,"1,426",99,331,3.3,3,171,5.6,100,12.6,"1,095",9,13,8,61.5%,9,8,1,24
1,Baltimore Ravens,6,104,397,5.1,"2,039",143,654,4.6,4,232,5.5,149,10.5,"1,385",8,13,10,76.9%,11,3,8,22
2,Denver Broncos,5,110,337,5.2,"1,749",130,553,4.3,3,193,5.8,127,10.2,"1,196",7,15,7,46.7%,5,3,2,14
3,New England Patriots,5,110,292,5.8,"1,695",141,596,4.2,1,143,7.3,94,12.4,"1,099",9,14,7,50%,10,6,4,8
4,Arizona Cardinals,6,112,403,5.2,"2,077",165,718,4.4,3,221,5.7,138,10.8,"1,359",9,24,10,41.7%,7,3,4,17


In [58]:
def_fantasy = fantasy_pts[fantasy_pts['Position'] == 'DST']

In [59]:
defense_df = pd.merge(defense, def_fantasy, 'left', left_on = 'Team', right_on = 'Player')

In [60]:
defense_df.head()

,Team_x,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,Player,Team_y,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Pittsburgh Steelers,5,94,294,4.9,"1,426",99,331,3.3,3,171,5.6,100,12.6,"1,095",9,13,8,61.5%,9,8,1,24,Pittsburgh Steelers,PIT,DST,54.0,8.0,13.0,7.0,10.0,8.0,18.0
1,Baltimore Ravens,6,104,397,5.1,"2,039",143,654,4.6,4,232,5.5,149,10.5,"1,385",8,13,10,76.9%,11,3,8,22,Baltimore Ravens,BAL,DST,70.0,15.0,15.0,1.0,6.0,26.0,7.0
2,Denver Broncos,5,110,337,5.2,"1,749",130,553,4.3,3,193,5.8,127,10.2,"1,196",7,15,7,46.7%,5,3,2,14,Denver Broncos,DEN,DST,28.0,2.0,5.0,1.0,6.0,NaN,14.0
3,New England Patriots,5,110,292,5.8,"1,695",141,596,4.2,1,143,7.3,94,12.4,"1,099",9,14,7,50%,10,6,4,8,New England Patriots,NE,DST,43.0,11.0,6.0,15.0,4.0,NaN,7.0
4,Arizona Cardinals,6,112,403,5.2,"2,077",165,718,4.4,3,221,5.7,138,10.8,"1,359",9,24,10,41.7%,7,3,4,17,Arizona Cardinals,ARI,DST,40.0,4.0,9.0,4.0,1.0,7.0,15.0


In [61]:
defense_df.drop(columns=['Player', 'Team_y', 'Position'], inplace=True)
defense_df.rename(columns={'Team_x': 'Team'}, inplace=True)

In [62]:
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Pittsburgh Steelers,5,94,294,4.9,"1,426",99,331,3.3,3,171,5.6,100,12.6,"1,095",9,13,8,61.5%,9,8,1,24,54.0,8.0,13.0,7.0,10.0,8.0,18.0
1,Baltimore Ravens,6,104,397,5.1,"2,039",143,654,4.6,4,232,5.5,149,10.5,"1,385",8,13,10,76.9%,11,3,8,22,70.0,15.0,15.0,1.0,6.0,26.0,7.0
2,Denver Broncos,5,110,337,5.2,"1,749",130,553,4.3,3,193,5.8,127,10.2,"1,196",7,15,7,46.7%,5,3,2,14,28.0,2.0,5.0,1.0,6.0,NaN,14.0
3,New England Patriots,5,110,292,5.8,"1,695",141,596,4.2,1,143,7.3,94,12.4,"1,099",9,14,7,50%,10,6,4,8,43.0,11.0,6.0,15.0,4.0,NaN,7.0
4,Arizona Cardinals,6,112,403,5.2,"2,077",165,718,4.4,3,221,5.7,138,10.8,"1,359",9,24,10,41.7%,7,3,4,17,40.0,4.0,9.0,4.0,1.0,7.0,15.0


In [63]:
defense_df.isnull().sum()

Team                         0
GP                           0
Ttl_Pts_Allowed              0
Ttl_Offense_Plays_Allowed    0
Yds_p_Play                   0
Ttl_Yds                      0
Rushing_Att                  0
Rushing_Yds                  0
Rushing_Yds_p_Att            0
Rushing_TDs                  0
Passing_Att                  0
Passing_Yds_p_Att            0
Completions                  0
Yds_p_Completion             0
Passing_Yds                  0
Passing_TDs                  0
RZ_Att                       0
RZ_TD                        0
RZ_Percent                   0
Ttl_Turnovers                0
Interceptions                0
Fumbles                      0
Sacks                        0
TTL                          0
Week_1                       0
Week_2                       0
Week_3                       0
Week_4                       0
Week_5                       4
Week_6                       4
dtype: int64

In [64]:
defense_df

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Pittsburgh Steelers,5,94,294,4.9,"1,426",99,331,3.3,3,171,5.6,100,12.6,"1,095",9,13,8,61.5%,9,8,1,24,54.0,8.0,13.0,7.0,10.0,8.0,18.0
1,Baltimore Ravens,6,104,397,5.1,"2,039",143,654,4.6,4,232,5.5,149,10.5,"1,385",8,13,10,76.9%,11,3,8,22,70.0,15.0,15.0,1.0,6.0,26.0,7.0
2,Denver Broncos,5,110,337,5.2,"1,749",130,553,4.3,3,193,5.8,127,10.2,"1,196",7,15,7,46.7%,5,3,2,14,28.0,2.0,5.0,1.0,6.0,NaN,14.0
3,New England Patriots,5,110,292,5.8,"1,695",141,596,4.2,1,143,7.3,94,12.4,"1,099",9,14,7,50%,10,6,4,8,43.0,11.0,6.0,15.0,4.0,NaN,7.0
4,Arizona Cardinals,6,112,403,5.2,"2,077",165,718,4.4,3,221,5.7,138,10.8,"1,359",9,24,10,41.7%,7,3,4,17,40.0,4.0,9.0,4.0,1.0,7.0,15.0
5,Miami Dolphins,6,113,375,5.8,"2,161",149,745,5,8,209,6.3,130,12,"1,416",7,18,13,72.2%,9,5,4,17,46.0,2.0,2.0,12.0,3.0,12.0,15.0
6,Los Angeles Rams,6,114,384,5,"1,911",150,654,4.4,5,214,5.4,139,10,"1,257",8,19,12,63.2%,6,4,2,20,38.0,4.0,7.0,4.0,11.0,12.0,0.0
7,Indianapolis Colts,6,115,358,4.8,"1,728",151,530,3.5,4,194,5.8,120,10.7,"1,198",7,15,10,66.7%,10,10,0,13,61.0,4.0,15.0,26.0,7.0,5.0,4.0
8,Chicago Bears,6,116,393,5.1,"2,023",159,678,4.3,5,219,5.7,125,11.5,"1,345",4,22,8,36.4%,8,5,3,15,38.0,3.0,12.0,4.0,2.0,6.0,11.0
9,Tampa Bay Buccaneers,6,122,366,4.6,"1,692",130,386,3,5,214,5.5,143,10.4,"1,306",7,17,10,58.8%,11,8,3,22,61.0,0.0,14.0,16.0,6.0,6.0,19.0


In [65]:
defense_df.fillna('0.0', inplace=True)

In [66]:
def_int = ['GP', 'Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Ttl_Yds', 'Rushing_Att', 'Rushing_Yds', 
           'Rushing_TDs', 'Passing_Att', 'Completions', 'Passing_Yds', 'Passing_TDs', 'RZ_Att', 'RZ_TD', 
           'Ttl_Turnovers', 'Interceptions', 'Fumbles', 'Sacks']

def_floats = ['Yds_p_Play', 'Rushing_Yds_p_Att', 'Passing_Yds_p_Att', 'Yds_p_Completion', 'TTL', 'Week_1', 
              'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6']

In [67]:
remove_comma(defense_df, def_int)

In [68]:
update_int_dtype(defense_df, def_int)

In [69]:
adjust_float(defense_df, def_floats)

In [70]:
update_float_dtype(defense_df, def_floats)

In [71]:
update_percent(defense_df, 'RZ_Percent')

In [72]:
rz = ['RZ_Percent']
update_float_dtype(defense_df, rz)

In [73]:
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Pittsburgh Steelers,5,94,294,4.9,1426,99,331,3.3,3,171,5.6,100,12.6,1095,9,13,8,61.5,9,8,1,24,54.0,8.0,13.0,7.0,10.0,8.0,18.0
1,Baltimore Ravens,6,104,397,5.1,2039,143,654,4.6,4,232,5.5,149,10.5,1385,8,13,10,76.9,11,3,8,22,70.0,15.0,15.0,1.0,6.0,26.0,7.0
2,Denver Broncos,5,110,337,5.2,1749,130,553,4.3,3,193,5.8,127,10.2,1196,7,15,7,46.7,5,3,2,14,28.0,2.0,5.0,1.0,6.0,0.0,14.0
3,New England Patriots,5,110,292,5.8,1695,141,596,4.2,1,143,7.3,94,12.4,1099,9,14,7,50.0,10,6,4,8,43.0,11.0,6.0,15.0,4.0,0.0,7.0
4,Arizona Cardinals,6,112,403,5.2,2077,165,718,4.4,3,221,5.7,138,10.8,1359,9,24,10,41.7,7,3,4,17,40.0,4.0,9.0,4.0,1.0,7.0,15.0


In [74]:
defense_df.dtypes

Team                          object
GP                             int64
Ttl_Pts_Allowed                int64
Ttl_Offense_Plays_Allowed      int64
Yds_p_Play                   float64
Ttl_Yds                        int64
Rushing_Att                    int64
Rushing_Yds                    int64
Rushing_Yds_p_Att            float64
Rushing_TDs                    int64
Passing_Att                    int64
Passing_Yds_p_Att            float64
Completions                    int64
Yds_p_Completion             float64
Passing_Yds                    int64
Passing_TDs                    int64
RZ_Att                         int64
RZ_TD                          int64
RZ_Percent                   float64
Ttl_Turnovers                  int64
Interceptions                  int64
Fumbles                        int64
Sacks                          int64
TTL                          float64
Week_1                       float64
Week_2                       float64
Week_3                       float64
W

## Kicking Stats

In [291]:
kicking.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS
0,Rodrigo Blankenship,K,IND,4,14,85.7,44,0-0,3-3,5-6,4-5,0-0,9,100.0,45
1,Randy Bullock,K,CIN,4,13,92.3,50,0-0,2-2,4-5,5-5,1-1,9,100.0,45
2,Ryan Succop,K,TB,5,11,81.8,46,0-0,2-2,5-5,2-3,0-1,15,93.3,41
3,Mason Crosby,K,GB,4,8,100.0,52,0-0,0-0,3-3,4-4,1-1,17,94.1,40
4,Daniel Carlson,K,LV,4,10,90.0,54,0-0,5-5,1-1,0-1,3-3,12,100.0,39


In [292]:
kicking_df = pd.merge(kicking, fantasy_pts, left_on = 'PLAYER', right_on = 'Player')
kicking_df.drop(columns = ['Player', 'Team', 'Position'], inplace = True)
kicking_df

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5
0,Rodrigo Blankenship,K,IND,4,14,85.7,44,0-0,3-3,5-6,4-5,0-0,9,100.0,45,60.0,8.0,15.0,12.0,14.0,11.0
1,Randy Bullock,K,CIN,4,13,92.3,50,0-0,2-2,4-5,5-5,1-1,9,100.0,45,55.0,10.0,13.0,12.0,17.0,3.0
2,Ryan Succop,K,TB,5,11,81.8,46,0-0,2-2,5-5,2-3,0-1,15,93.3,41,43.0,5.0,7.0,9.0,8.0,14.0
3,Mason Crosby,K,GB,4,8,100.0,52,0-0,0-0,3-3,4-4,1-1,17,94.1,40,46.0,12.0,11.0,16.0,7.0,NaN
4,Daniel Carlson,K,LV,4,10,90.0,54,0-0,5-5,1-1,0-1,3-3,12,100.0,39,56.0,12.0,12.0,8.0,13.0,11.0
5,Justin Tucker,K,BAL,4,8,100.0,47,0-0,2-2,2-2,4-4,0-0,14,100.0,38,52.0,9.0,16.0,9.0,8.0,10.0
6,Joey Slye,K,CAR,4,11,90.9,47,0-0,5-5,3-3,2-3,0-0,9,77.8,37,50.0,12.0,5.0,15.0,7.0,11.0
7,Wil Lutz,K,NO,4,6,100.0,45,0-0,2-2,3-3,1-1,0-0,15,100.0,33,49.0,10.0,6.0,13.0,5.0,15.0
8,Jason Sanders,K,MIA,4,9,100.0,52,0-0,2-2,1-1,5-5,1-1,6,100.0,33,62.0,4.0,11.0,7.0,18.0,22.0
9,Stephen Gostkowski,K,TEN,3,12,75.0,55,0-0,1-1,3-3,1-4,4-4,7,71.4,32,47.0,4.0,12.0,25.0,NaN,6.0


In [293]:
dash = kicking_df['LNG'][35]
dash

'—'

In [294]:
kicking_df['LNG'].value_counts()

52    4
50    4
47    3
44    3
54    3
—     3
56    2
41    2
46    2
55    2
43    2
49    1
35    1
39    1
45    1
23    1
58    1
Name: LNG, dtype: int64

In [295]:
kicking_df.fillna('0.0', inplace = True)

In [296]:
kick_int = ['GP', 'FGM_A', 'LNG', 'XPM_A', 'PTS']

kick_float = ['FG%', 'XP%', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5']

In [297]:
replace_dash(kicking_df, kick_int, dash)

In [298]:
replace_dash(kicking_df, kick_float, dash)

In [299]:
update_int_dtype(kicking_df, kick_int)

In [300]:
adjust_float(kicking_df, kick_float)

In [301]:
update_float_dtype(kicking_df, kick_float)

In [302]:
kicking_df.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5
0,Rodrigo Blankenship,K,IND,4,14,85.7,44,0-0,3-3,5-6,4-5,0-0,9,100.0,45,60.0,8.0,15.0,12.0,14.0,11.0
1,Randy Bullock,K,CIN,4,13,92.3,50,0-0,2-2,4-5,5-5,1-1,9,100.0,45,55.0,10.0,13.0,12.0,17.0,3.0
2,Ryan Succop,K,TB,5,11,81.8,46,0-0,2-2,5-5,2-3,0-1,15,93.3,41,43.0,5.0,7.0,9.0,8.0,14.0
3,Mason Crosby,K,GB,4,8,100.0,52,0-0,0-0,3-3,4-4,1-1,17,94.1,40,46.0,12.0,11.0,16.0,7.0,0.0
4,Daniel Carlson,K,LV,4,10,90.0,54,0-0,5-5,1-1,0-1,3-3,12,100.0,39,56.0,12.0,12.0,8.0,13.0,11.0


In [303]:
kicking_df.dtypes

PLAYER       object
POSITION     object
TEAM         object
GP            int64
FGM_A         int64
FG%         float64
LNG           int64
_1_19        object
_20_29       object
_30_39       object
_40_49       object
OVER50       object
XPM_A         int64
XP%         float64
PTS           int64
TTL         float64
Week_1      float64
Week_2      float64
Week_3      float64
Week_4      float64
Week_5      float64
dtype: object

## Pickle Cleaned DataFrames

In [304]:
offense.to_pickle('players')
team_names.to_pickle('long_names')
defense_df.to_pickle('defense')
kicking_df.to_pickle('kicking')